In [1]:
import pandas as pd
import os 
file = "/data/steam_games.csv"
file = os.getcwd() + file
raw_steam_data = pd.read_csv(file, nrows=8000)

In [2]:
raw_steam_data['name'].size #40833 games
names = raw_steam_data['name'] 
names.size #40833 names (possibly not unique) 
types = raw_steam_data['types']
#types[types.duplicated(keep=False)].size

#example of duplicate names -> unsuitable for unique id
for i in range(0, raw_steam_data['name'].size):
    if "The Deer" == raw_steam_data['name'][i]:
        print(raw_steam_data.iloc[i])
        print(i)

In [3]:
#purging unwanted data
purged_steam_data = raw_steam_data.drop(columns=['types', 'desc_snippet', 'achievements','game_description', 'minimum_requirements', 'recommended_requirements'])
purged_steam_data.iloc[5]

url               https://store.steampowered.com/bundle/5699/Gra...
name                     Grand Theft Auto V: Premium Online Edition
recent_reviews                                                  NaN
all_reviews                                                     NaN
release_date                                                    NaN
developer                                            Rockstar North
publisher                                            Rockstar Games
popular_tags                                                    NaN
game_details      Single-player,Multi-player,Downloadable Conten...
languages         English, French, Italian, German, Spanish - Sp...
genre                                              Action,Adventure
mature_content                                                  NaN
original_price                                                  NaN
discount_price                                               $35.18
Name: 5, dtype: object

In [4]:
import re
import math
amount_p = re.compile(r'\d+(,\d+)*')
value_p = re.compile(r'\d+')

def process_review(review):
    #print(review)
    if isinstance(review, str) :
        str_num = amount_p.search(review).group()
        nums = value_p.findall(str_num)
        amt = 0
        if 1 < len(nums):
            for i in range(0, len(nums) - 1):
                prev = nums[i + 1]
                power = len(prev)
                prev = int(prev)
                amt += int(nums[i])*(10**power) + prev
                
        else:
            amt = nums[0]
        
        all_nums = value_p.findall(review)
        if 2 < len(all_nums):
            val = all_nums[len(nums)]
        else:
            val = -1
            
        return [amt, val]
    else: 
        return []
    
def get_element(nums, index):
    if index < len(nums):
        return int(nums[index])
    else:
        return 0

recent_review_numbers = purged_steam_data.apply(lambda x: process_review(x['recent_reviews']), axis=1)
recent_review_numbers


purged_steam_data['amt_rec_revs'] = recent_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['rec_rev_score'] = recent_review_numbers.apply(lambda x: get_element(x, 1))

alltime_review_numbers = purged_steam_data.apply(lambda x: process_review(x['all_reviews']), axis=1)

purged_steam_data['amt_all_revs'] = alltime_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['all_rev_score'] = alltime_review_numbers.apply(lambda x: get_element(x, 1))

purged_steam_data = purged_steam_data.drop(columns=['recent_reviews','all_reviews'])

#keys for new columns:
#'amt_rec_revs' -> the amount of recent reviews
#'rec_rev_score' -> the percentage of the recent reviews
#'amt_all_revs' -> the amount of all reviews
#'all_rev_score' - > the perceent of all reviews


In [5]:
import functools 

def handle_prices(price):
    if isinstance(price, str):
        prices = value_p.findall(price)
        if len(prices):
            price = float(prices[0]) 
            if 1 < len(prices):
                price += float(prices[1])/100
            return price
        else:
            return 0
    else:
        return -1 #return nan

purged_steam_data['original_price'] = purged_steam_data.apply(lambda x: handle_prices(x['original_price']), axis=1)
purged_steam_data['discount_price'] = purged_steam_data.apply(lambda x: handle_prices(x['discount_price']), axis=1)


#value_p.findall(purged_steam_data['original_price'][4])

In [6]:
purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,19.99,14.99,554,89,42550,92
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",NaN,39.99,-1.00,166,54,7030,71
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,44.99,-1.00,932,57,167115,61
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,0.00,-1.00,287,54,11481,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,"Kalypso Media Digital,Kalypso Media Digital","Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",NaN,9.99,-1.00,0,0,134,85
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,"Choice of Games,Choice of Games","Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",NaN,501.87,424.72,0,0,112,88
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,"Winter Wolves,Winter Wolves","Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",NaN,475.60,367.96,0,0,58,81
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,"SCS Software,SCS Software","Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",NaN,1.99,-1.00,0,0,96,89


In [7]:
from collections import Counter 

def remove_duplicates(input): 
    if (type(input) == str):
        # split input string separated by space 
        input = input.split(",") 

        # joins two adjacent elements in iterable way 
        for i in range(0, len(input)): 
            input[i] = "".join(input[i]) 

        # now create dictionary using counter method 
        # which will have strings as key and their 
        # frequencies as value 
        UniqW = Counter(input) 

        # joins two adjacent elements in iterable way 
        s = " ".join(UniqW.keys()) 
        return s
    else:
        s = ""
        return s


purged_steam_data['publisher'] = purged_steam_data.apply(lambda x: remove_duplicates(x['publisher']), axis=1)

purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,Bethesda Softworks,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,19.99,14.99,554,89,42550,92
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,Paradox Interactive,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",NaN,39.99,-1.00,166,54,7030,71
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,44.99,-1.00,932,57,167115,61
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,0.00,-1.00,287,54,11481,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,Kalypso Media Digital,"Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",NaN,9.99,-1.00,0,0,134,85
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,Choice of Games,"Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",NaN,501.87,424.72,0,0,112,88
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,Winter Wolves,"Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",NaN,475.60,367.96,0,0,58,81
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,SCS Software,"Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",NaN,1.99,-1.00,0,0,96,89


In [8]:
#find games that have no popular tags and change to empty to string 

def remove_nans(input):
    if (isinstance(input, str)):
        return input
    else:
        return ""

purged_steam_data['popular_tags'] = purged_steam_data.apply(lambda x: remove_nans(x['popular_tags']), axis=1)

In [9]:
#find number of days since games release relative to today
import datetime
release = datetime.datetime.strptime("May 12, 2016", '%b %d, %Y')
today = datetime.datetime.now()
abs((release - today).days)

1409

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tif_matrix = vectorizer.fit_transform(purged_steam_data['popular_tags'])
for name in vectorizer.get_feature_names():
    print(name)
print(vectorizer.get_feature_names())
print(tif_matrix.shape)
#tif_matrix

1980s
1990
2d
360
3d
40k
4x
5d
6dof
abstract
access
action
addictive
adventure
agriculture
aliens
alternate
america
and
animation
anime
apocalyptic
arcade
arena
artificial
arts
assassin
asymmetric
asynchronous
atmospheric
attack
atv
audio
automation
awkward
base
baseball
based
basketball
batman
battle
beat
beautiful
benchmark
bikes
blood
bmx
board
book
bowling
builder
building
bullet
campaign
capitalism
card
cartoon
cartoony
casual
cats
character
chess
choices
choose
cinematic
city
class
classic
click
clicker
co
cold
colorful
combat
comedy
comic
competitive
conspiracy
content
control
controller
controls
conversation
crafting
crawler
crime
croft
crowdfunded
crpg
cult
customization
cute
cyberpunk
cycling
dark
dating
death
defense
demons
design
destruction
detective
development
difficult
dinosaurs
diplomacy
documentary
dog
down
dragons
drama
drawn
driving
dungeon
dungeons
dynamic
dystopian
early
economy
editing
editor
education
em
emotional
endings
epic
episodic
events
experience
experime

In [11]:
#find games that have no mature content and change to empty to string 
purged_steam_data['mature_content'] = purged_steam_data.apply(lambda x: remove_nans(x['mature_content']), axis=1)

In [12]:
from sklearn.metrics.pairwise import linear_kernel 
cosine_similarities = linear_kernel(tif_matrix, tif_matrix) 
results = {}
for idx, row in purged_steam_data.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] #get top 99 most common games
   similar_items = [(cosine_similarities[idx][i], purged_steam_data['url'][i]) for i in similar_indices] 
   results[row['url']] = similar_items[1:]

In [13]:
results[purged_steam_data['url'][1]]
#purged_steam_data['url'][0] ['https://store.steampowered.com/app/782330/DOOM_Eternal/']
#['https://store.steampowered.com/app/718560/Scythe_Digital_Edition/']

[(0.8233990503997944,
  'https://store.steampowered.com/app/813820/Realm_Royale/'),
 (0.7499814574458284,
  'https://store.steampowered.com/app/15120/Tom_Clancys_Rainbow_Six_Vegas_2/'),
 (0.7497570885200636,
  'https://store.steampowered.com/app/460930/Tom_Clancys_Ghost_Recon_Wildlands/'),
 (0.7211355691948168,
  'https://store.steampowered.com/app/555440/Deathgarden_BLOODHARVEST/'),
 (0.7044567029432889,
  'https://store.steampowered.com/app/433850/Z1_Battle_Royale/'),
 (0.6880425437439217,
  'https://store.steampowered.com/app/893520/Creative_Destruction/'),
 (0.6879347540598457,
  'https://store.steampowered.com/app/755790/Ring_of_Elysium/'),
 (0.6827555477703714,
  'https://store.steampowered.com/app/235600/Tom_Clancys_Splinter_Cell_Blacklist/'),
 (0.6668164754299969,
  'https://store.steampowered.com/app/104900/ORION_Prelude/'),
 (0.6596424882738731,
  'https://store.steampowered.com/app/215470/Primal_Carnage/'),
 (0.6578536091396916, 'https://store.steampowered.com/app/222880/Ins

In [14]:
results['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

[(0.7920010868868157,
  'https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/'),
 (0.7384038796309083,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.6929467233601276,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.6929467233601276,
  'https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/'),
 (0.6301359854276765,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.6202663629698706,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.6003324891231278,
  'https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/'),
 (0.5659228328161767,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.556339297470859, 'https://store.steampowered.com/app/551770/ECHO/'),
 (0.5422135720823295, 'https://store.steampowered.com/app/253

In [18]:
#cosine model function
def cos_recommend(url, no_games):
    for rec in results[url][:no_games]:
        print("Game URL: " + rec[1])
        print("Recomendation Score: " + str(rec[0]))

cos_recommend('https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/', 10)

Game URL: https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/
Recomendation Score: 0.7920010868868157
Game URL: https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/
Recomendation Score: 0.7384038796309083
Game URL: https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/
Recomendation Score: 0.6929467233601276
Game URL: https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/
Recomendation Score: 0.6929467233601276
Game URL: https://store.steampowered.com/app/288470/Fable_Anniversary/
Recomendation Score: 0.6301359854276765
Game URL: https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/
Recomendation Score: 0.6202663629698706
Game URL: https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/
Recomendation Score: 0.6003324891231278
Game URL: https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters

In [80]:
#'age' no of days since game added to store 
import dateparser
today = datetime.datetime.today()

release = dateparser.parse(purged_steam_data['release_date'][0])
age = []

for i in range(0, len(purged_steam_data)):
    str_release = purged_steam_data['release_date'][i]
    print(str_release)
    if((type(str_release) == str) and 
       (str_release != 'TBA' )):
        if(str_release.split()[0] == 'Summer'):
            year = str_release.split()[1]
            print("Jun 20, " + year)
            release = dateparser.parse("Jun 20, " + year)
            age.append(abs((release - today).days))
        else:
            release = dateparser.parse(purged_steam_data['release_date'][i])     
            age.append(abs((release - today).days))
    else:
        age.append(abs((release - today).days))
 

purged_steam_data['age'] = age
#abs((release - today).days)
purged_steam_data


May 12, 2016
Dec 21, 2017
Apr 24, 2018
Dec 13, 2018
May 6, 2003
nan
Mar 7, 2019
Jul 22, 2016
Dec 12, 2017
May 31, 2019
Apr 19, 2018
Sep 25, 2017
May 5, 2015
Jul 27, 2017
Jul 25, 2018
nan
Mar 16, 2017
Feb 7, 2019
Apr 5, 2018
Apr 20, 2017
Nov 9, 2010
Apr 9, 2019
May 21, 2019
Aug 31, 2016
Dec 2, 2008
nan
Oct 10, 2007
Oct 23, 2015
Mar 27, 2018
Mar 23, 2018
Mar 28, 2016
Aug 8, 2017
Apr 18, 2016
Aug 1, 2018
Aug 21, 2017
Nov 1, 2005
Sep 2, 2008
Sep 26, 2018
Feb 7, 2012
nan
Feb 28, 2011
Jun 13, 2019
Nov 1, 2004
Oct 13, 2014
nan
Jan 11, 2019
Feb 16, 2006
Sep 14, 2017
Feb 15, 2016
Oct 13, 2015
Jun 5, 2019
Jan 24, 2019
Apr 2, 2015
Sep 16, 2008
Oct 9, 2017
Apr 19, 2017
Jun 6, 2012
nan
Jun 5, 2019
Oct 27, 2010
Mar 26, 2015
Aug 15, 2018
May 31, 2017
nan
Mar 7, 2018
Jun 5, 2018
nan
Dec 12, 2017
nan
Nov 15, 2006
Jan 10, 2019
May 12, 2016
Oct 18, 2018
Sep 26, 2012
nan
Nov 2, 2017
Oct 30, 2017
Oct 8, 2015
Dec 19, 2008
Jun 11, 2019
Aug 11, 2015
nan
Nov 30, 2015
Mar 25, 2013
May 15, 2014
Jun 25, 2012
nan


Dec 14, 2016
Mar 11, 2016
Oct 28, 2008
Nov 15, 2018
Sep 11, 2006
Oct 17, 2013
May 29, 2019
nan
Oct 16, 2008
Aug 18, 2014
Dec 11, 2018
Nov 30, 2017
May 18, 2017
Jun 5, 2018
Sep 18, 2014
Jun 12, 2012
May 31, 2019
Oct 23, 2018
Jun 16, 2017
Aug 29, 2018
Mar 4, 2016
Jan 19, 2015
Nov 26, 2014
Aug 9, 2016
Jan 29, 2015
Nov 20, 2014
Jul 20, 2017
Dec 7, 2017
Mar 29, 2017
Feb 23, 2018
nan
Aug 1, 2017
Dec 1, 2017
nan
Nov 17, 2008
Apr 18, 2018
nan
Feb 27, 2018
Feb 27, 2012
Feb 18, 2016
Feb 9, 2016
May 28, 2019
nan
Jun 23, 2015
Dec 10, 2015
May 23, 2019
nan
Jan 1, 2006
Aug 9, 2018
Jun 6, 2019
Dec 15, 2015
Jun 13, 2019
nan
Feb 25, 2016
nan
Apr 30, 2019
Jun 6, 2005
Jun 21, 2016
Mar 4, 2009
Dec 4, 2018
Oct 13, 2009
Aug 17, 2018
Oct 15, 2012
Feb 21, 2018
Sep 15, 2015
Feb 18, 2016
Feb 6, 2018
Aug 6, 2018
Oct 11, 2018
Jun 20, 2017
nan
nan
nan
nan
Mar 6, 2017
Aug 2, 2016
Mar 2020
Nov 23, 2018
nan
Mar 3, 2017
Oct 10, 2016
Aug 26, 2016
Mar 21, 2018
Mar 5, 2008
May 30, 2019
Sep 3, 2019
Apr 28, 2015
Oct 2, 201

Dec 14, 2018
Sep 12, 2016
Nov 7, 2017
Apr 4, 2016
Mar 22, 2011
Jun 6, 2013
nan
Feb 3, 2011
Dec 19, 2008
Jun 3, 2016
Sep 28, 2017
Apr 5, 2016
Oct 27, 2014
Nov 29, 2018
Sep 1, 2017
nan
Jun 23, 2009
Aug 23, 2018
Jul 26, 2017
Oct 1, 2009
nan
Sep 7, 2012
Mar 19, 2019
Oct 4, 2016
Jan 18, 2016
nan
Sep 26, 2018
Sep 5, 2016
Apr 11, 2018
Aug 28, 2018
Dec 22, 2018
Feb 13, 2015
Oct 4, 2017
Aug 14, 2018
Sep 8, 2017
Oct 31, 1999
nan
May 14, 2015
Jan 30, 2014
Nov 20, 2001
Jun 29, 2010
May 2, 2019
Aug 29, 2017
Jul 5, 2016
May 31, 2018
Oct 18, 2018
Nov 8, 2018
Mar 4, 2011
Mar 24, 2003
nan
Jul 20, 2017
nan
Nov 3, 2016
Jul 2014
May 13, 2003
Mar 15, 2018
Feb 26, 2019
Apr 26, 2019
Jun 5, 2018
May 18, 2017
Jan 19, 2015
Jan 30, 2019
nan
Apr 25, 2018
Mar 13, 2018
nan
Feb 18, 2019
Sep 20, 2018
Feb 27, 2017
Apr 23, 2015
Nov 19, 2015
Sep 22, 2015
May 14, 2015
Nov 17, 2015
nan
Dec 20, 2017
nan
Oct 4, 1996
Feb 28, 2018
Feb 12, 2019
Aug 17, 2018
Nov 18, 2016
Jun 6, 2014
Sep 23, 2014
Jun 6, 2017
nan
May 20, 2019
Nov

Jun 5, 2007
Nov 20, 2015
Mar 29, 1996
nan
Jul 13, 2018
Jan 8, 2016
Nov 24, 2009
Mar 21, 2014
Dec 12, 2006
Jul 3, 2015
Nov 6, 2015
Feb 7, 2017
Jun 6, 2019
Sep 21, 2017
Jan 25, 2019
Oct 11, 2011
Oct 11, 2018
Oct 14, 2016
Oct 29, 1997
Apr 5, 2016
Jul 10, 2015
Jun 21, 2016
Aug 6, 2014
Jun 21, 2018
Feb 25, 2014
May 29, 2015
May 20, 2015
Nov 16, 2011
Mar 28, 2017
Jan 7, 2019
May 21, 2019
Feb 28, 1998
Aug 7, 2017
Nov 22, 2016
Dec 15, 2011
Jan 21, 2019
Aug 28, 2015
Oct 21, 2013
Sep 19, 2018
Aug 30, 2016
May 30, 2019
Mar 15, 2018
Jun 30, 2017
nan
Apr 2, 2015
Jun 1, 2015
Sep 13, 2018
May 8, 2018
Dec 8, 2018
Apr 4, 2019
Jul 6, 2017
nan
Mar 30, 2017
Jul 1, 2013
Oct 17, 1997
Mar 26, 2019
Sep 23, 2010
Nov 3, 2011
Apr 13, 2016
Mar 8, 2018
Dec 12, 2014
Mar 1, 2017
nan
Jul 25, 2017
nan
Dec 14, 2017
Oct 30, 2014
nan
Jun 21, 2018
Sep 21, 2018
Aug 25, 2016
Dec 19, 2016
Sep 28, 2017
Mar 15, 2001
Aug 31, 2016
nan
Apr 30, 2019
Nov 5, 2018
Nov 22, 2016
Nov 17, 2011
Jul 23, 2015
Oct 26, 2018
Apr 5, 2017
Sep 7,

Nov 10, 2016
May 16, 2013
Aug 4, 2015
Sep 28, 2017
May 17, 2018
May 6, 2009
Jul 24, 2007
Nov 1, 2018
Mar 3, 2014
nan
Jan 18, 2018
Mar 29, 2012
Apr 14, 2019
Jun 6, 2014
Jul 4, 2013
Mar 22, 2018
Feb 7, 2014
Aug 22, 2003
Jul 19, 2016
Aug 26, 2010
Jan 21, 2017
nan
Jun 24, 2010
Aug 4, 2011
Jul 25, 2018
Sep 28, 2010
Sep 29, 2016
Mar 17, 2016
Sep 8, 2016
Jul 26, 2018
Apr 12, 2018
Apr 11, 2012
Aug 28, 1998
Mar 8, 2018
Feb 1, 2019
Nov 13, 2001
May 25, 2017
May 13, 2019
Apr 21, 2016
Aug 11, 2016
Sep 13, 2018
Jun 30, 1995
Apr 26, 2019
Oct 30, 2012
Sep 20, 2017
Dec 24, 2018
Mar 29, 2011
Dec 7, 2005
Mar 22, 2019
Sep 14, 2017
Dec 17, 2018
Jan 23, 2014
Sep 21, 2018
Sep 20, 2013
Apr 11, 2017
May 12, 2017
Mar 3, 2017
Mar 3, 2016
Jan 5, 2018
Mar 14, 2011
Feb 22, 2019
Jan 18, 2018
Mar 31, 2017
Sep 26, 2018
Aug 1, 2006
Sep 29, 2017
nan
Dec 9, 2014
Dec 18, 2018
nan
May 11, 2015
Jan 30, 2018
May 30, 2013
nan
Nov 30, 2016
Nov 21, 2016
Jun 15, 2018
Apr 18, 2017
Apr 28, 2019
Apr 21, 2017
Dec 19, 2018
Feb 27, 2

...

Sep 6, 2018
Feb 19, 2015
Oct 18, 2018
Apr 19, 2019
Mar 1, 1997
May 11, 2018
Jul 16, 2018
Dec 18, 2018
Oct 22, 2018
May 3, 2017
Sep 27, 2017
Dec 6, 2018
Dec 11, 2012
nan
Apr 18, 2018
Oct 15, 2010
Apr 20, 2004
Sep 13, 2005
Jun 1, 2015
Oct 19, 2017
Apr 2, 2019
Dec 13, 2018
Feb 8, 2017
Dec 6, 2016
Dec 14, 2017
Nov 23, 2000
nan
Mar 5, 2019
Jun 13, 2019
Nov 28, 2016
Feb 29, 1996
May 17, 2019
nan
nan
Oct 26, 2017
Oct 20, 2017
Oct 6, 2016
nan
Nov 20, 2008
May 23, 2006
Jun 12, 2019
Feb 29, 1996
Oct 3, 2008
Aug 11, 2017
Mar 27, 2017
Feb 19, 2019
nan
Jan 1, 1993
Oct 18, 2018
Aug 26, 2014
Sep 26, 2017
May 10, 2018
Sep 9, 2016
Mar 17, 2017
nan
Nov 14, 2005
Mar 6, 2018
Apr 4, 2019
Jan 20, 2019
Apr 16, 2015
Dec 13, 2018
Aug 25, 2015
Nov 18, 1999
May 11, 2017
Feb 22, 2017
Mar 14, 2019
nan
Feb 18, 2016
nan
Aug 30, 2006
Jul 20, 2017
nan
nan
nan
Apr 30, 2009
nan
Jul 26, 2017
Dec 26, 2018
Mar 20, 2009
Apr 4, 2016
Feb 22, 2011
nan
Jan 10, 2019
Sep 1, 2017
nan
Apr 17, 2019
Jun 3, 2019
Apr 19, 2019
Dec 13, 2

nan
Feb 26, 2019
May 14, 2019
Feb 14, 2007
Sep 10, 2013
Oct 3, 2017
May 29, 2018
May 9, 2019
Jan 28, 2019
Mar 7, 2019
Jul 1, 2018
nan
Dec 21, 2017
May 27, 2019
May 1, 2019
Jul 28, 2016
Feb 14, 2013
Aug 18, 2015
nan
Mar 11, 2017
Jan 5, 2018
Apr 24, 2019
nan
Feb 15, 2019
nan
Apr 7, 2017
Mar 28, 2019
Dec 21, 2017
Dec 7, 2016
nan
Mar 11, 2016
Sep 28, 2016
Feb 28, 2018
Jul 31, 1999
nan
Mar 22, 2019
nan
Oct 3, 2003
nan
Mar 8, 2017
Sep 26, 2013
nan
Oct 12, 2017
Sep 13, 2018
Sep 5, 2016
Feb 10, 2009
Jan 28, 2019
May 15, 2018
Dec 1, 2016
Feb 24, 2017
Jan 16, 2019
Jan 22, 2019
Jul 28, 2017
nan
Oct 17, 2018
Sep 21, 2012
nan
Feb 26, 2019
Dec 13, 2018
Feb 27, 2015
Mar 26, 2015
Mar 18, 2019
nan
nan
Feb 19, 2019
Apr 24, 2014
Apr 4, 2019
Nov 22, 2018
Sep 1, 2016
nan
Dec 7, 2017
nan
Feb 14, 2019
Feb 18, 2019
Apr 25, 2014
May 28, 2015
Feb 13, 2017
Jun 19, 2002
Jun 22, 2016
Apr 19, 2012
Jun 29, 2017
Sep 20, 2016
Mar 28, 2018
Nov 25, 2000
Jul 19, 2017
May 23, 2019
Mar 10, 2003
Sep 12, 2006
Apr 4, 2014
Mar

Nov 15, 2017
Apr 18, 2016
Jun 11, 2018
nan
Jul 21, 2017
nan
Feb 19, 2018
nan
nan
nan
nan
Aug 15, 2017
Mar 13, 2018
Aug 10, 2018
Feb 12, 2018
nan
Jun 14, 2018
Feb 13, 2018
nan
Oct 5, 2017
Jan 26, 2017
Dec 21, 2017
Nov 17, 2018
Nov 30, 2017
Dec 19, 2016
Jan 1, 1992
Aug 23, 2018
Jun 10, 2017
Aug 28, 2017
Oct 20, 2017
Feb 13, 2017
Dec 4, 2014
Nov 21, 2016
Nov 10, 2016
Nov 9, 2017
Aug 24, 2015
Aug 19, 2016
May 27, 2019
Nov 2, 2015
May 30, 2017
Jun 26, 2015
nan
Sep 13, 2017
Nov 2, 2017
Nov 24, 2017
Jan 9, 2018
Nov 22, 2018
Sep 6, 2018
Sep 8, 2017
Aug 9, 2017
Apr 26, 2018
Apr 10, 2017
Feb 20, 2018
Dec 21, 2016
Jul 17, 2014
Oct 27, 2017
Sep 4, 2018
Aug 17, 2017
Mar 28, 2018
Jul 27, 2015
Jul 19, 2017
Sep 4, 2015
Dec 10, 2018
Nov 20, 2018
Sep 15, 2017
May 8, 2017
Oct 9, 2017
May 5, 2018
Jan 11, 2017
Sep 20, 2018
Nov 12, 2018
Jul 12, 2017
Jun 2, 2019
Nov 21, 2018
Jul 31, 2018
Mar 7, 2018
nan
Dec 18, 2017
Nov 16, 2015
Oct 3, 2018
Mar 15, 2018
Apr 13, 2018
nan
Nov 25, 2016
Jan 5, 2019
Jul 20, 2018


Feb 8, 2011
Feb 23, 2011
Dec 10, 2010
Dec 3, 2010
Jan 22, 2018
May 30, 2018
Feb 7, 2019
nan
Nov 19, 2018
Nov 24, 2017
Jan 10, 2007
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Sep 19, 2014
Mar 4, 2015
nan
Dec 11, 2015
Nov 4, 2014
Jun 25, 2015
Jan 27, 2015
nan
Jan 27, 2015
Sep 30, 2014
Mar 3, 2015
Dec 20, 2013
Oct 20, 2016
Sep 15, 2014
Dec 10, 2014
Sep 23, 2014
nan
May 29, 2007
nan
May 12, 2015
Apr 13, 2012
Feb 28, 2018
Apr 2, 2018
Nov 13, 2018
May 31, 2018
Jul 30, 2018
Feb 13, 2018
Mar 8, 2018
nan
May 20, 2018
nan
nan
Sep 17, 2009
Feb 18, 2010
Sep 3, 2009
nan
Oct 15, 2009
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Sep 1, 2017
Apr 18, 2014
nan
Dec 16, 2010
Oct 25, 2013
Sep 30, 2014
nan
May 7, 2015
nan
Jul 16, 2015
nan
Sep 4, 2018
May 2, 2015
Dec 3, 2015
Mar 26, 2013
Jul 2, 2014
Jul 2, 2014
Jun 27, 2014
Oct 24, 2003
nan
Sep 20, 2013
Feb 13, 2015
Jun 6, 2014
May 23, 2014
May 16, 2014
Oct 9, 2013
Jun 9, 2015
Jun 15, 2011
Feb 23, 2015
Apr 5, 2016
Mar 2, 2015
Feb 

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score,age
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,Bethesda Softworks,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,,19.99,14.99,554,89,42550,92,1409
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49,821
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,Paradox Interactive,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",,39.99,-1.00,166,54,7030,71,697
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",,44.99,-1.00,932,57,167115,61,464
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",,0.00,-1.00,287,54,11481,74,6164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,Kalypso Media Digital,"Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",,9.99,-1.00,0,0,134,85,1670
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,Choice of Games,"Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",,501.87,424.72,0,0,112,88,2108
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,Winter Wolves,"Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",,475.60,367.96,0,0,58,81,2347
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,SCS Software,"Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",,1.99,-1.00,0,0,96,89,2076


In [91]:
import numpy as np

w30 = np.zeros(8000)
wAll = np.ones(8000)
for i in range(0, len(purged_steam_data)):
    age = purged_steam_data['age'][i]
    if (30 < age):
        w30[i] = ((age - 30)/30)
        
total = w30 + 1
w30 = w30/total
wAll = wAll/total

purged_steam_data['w30'] = w30
purged_steam_data['wAll'] = wAll

#keys for new columns:
#'w30' -> weight of the 30 days recommendations 
#'wAll' -> weight of all recommendations 

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score,age,w30,wAll
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,Bethesda Softworks,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,,19.99,14.99,554,89,42550,92,1409,0.978708,0.021292
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49,821,0.963459,0.036541
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,Paradox Interactive,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",,39.99,-1.00,166,54,7030,71,697,0.956958,0.043042
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",,44.99,-1.00,932,57,167115,61,464,0.935345,0.064655
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",,0.00,-1.00,287,54,11481,74,6164,0.995133,0.004867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,Kalypso Media Digital,"Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",,9.99,-1.00,0,0,134,85,1670,0.982036,0.017964
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,Choice of Games,"Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",,501.87,424.72,0,0,112,88,2108,0.985769,0.014231
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,Winter Wolves,"Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",,475.60,367.96,0,0,58,81,2347,0.987218,0.012782
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,SCS Software,"Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",,1.99,-1.00,0,0,96,89,2076,0.985549,0.014451


In [93]:
#
psd = purged_steam_data
purged_steam_data['weighted_revs'] = (psd['w30'] * (psd['rec_rev_score']/100)) + (psd['wAll'] * (psd['all_rev_score']/100)) 

purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score,age,w30,wAll,weighted_revs
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,Bethesda Softworks,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,,19.99,14.99,554,89,42550,92,1409,0.978708,0.021292,0.890639
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49,821,0.963459,0.036541,0.490000
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,Paradox Interactive,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",,39.99,-1.00,166,54,7030,71,697,0.956958,0.043042,0.547317
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",,44.99,-1.00,932,57,167115,61,464,0.935345,0.064655,0.572586
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",,0.00,-1.00,287,54,11481,74,6164,0.995133,0.004867,0.540973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,Kalypso Media Digital,"Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",,9.99,-1.00,0,0,134,85,1670,0.982036,0.017964,0.015269
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,Choice of Games,"Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",,501.87,424.72,0,0,112,88,2108,0.985769,0.014231,0.012524
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,Winter Wolves,"Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",,475.60,367.96,0,0,58,81,2347,0.987218,0.012782,0.010354
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,SCS Software,"Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",,1.99,-1.00,0,0,96,89,2076,0.985549,0.014451,0.012861


In [148]:
psd = purged_steam_data

weighted_results = {}
'''
for url in purged_steam_data['url']:
    games = []
    for game in results[url]:
        games.append((game[1], game[0] * psd.loc[psd['url'] == game[1]]['weighted_revs']))
    
    weighted_results[url] = games'''

similarities = cosine_similarities
for idx, row in purged_steam_data.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] #get top 99 most common games
   for i in similar_indices:
        similarities[idx][i] = cosine_similarities[idx][i] * purged_steam_data['weighted_revs'][i]
        
   similar_indices = similarities[idx].argsort()[:-100:-1]
   similar_items = [(similarities[idx][i], purged_steam_data['url'][i]) for i in similar_indices] 
   
   #similar_items.append((similarities.argsort(), purged_steam_data['url'][i]))
   #similar_items = [(cosine_similarities[idx][i] * purged_steam_data['weighted_revs'][i], purged_steam_data['url'][i]) for i in similar_indices] 
   weighted_results[row['url']] = similar_items[1:]
   


#print(psd.loc[psd['url'] == 'https://store.steampowered.com/app/379720/DOOM/']['weighted_revs'])
weighted_results['https://store.steampowered.com/app/379720/DOOM/']
#weighted_results['https://store.steampowered.com/app/379720/DOOM/']

[(0.4835135391347939,
  'https://store.steampowered.com/app/214870/Painkiller_Hell__Damnation/'),
 (0.45630145723288024, 'https://store.steampowered.com/app/519860/DUSK/'),
 (0.4328169563530325,
  'https://store.steampowered.com/app/13230/Unreal_Tournament_2004_Editors_Choice_Edition/'),
 (0.40905127873972325, 'https://store.steampowered.com/app/2320/QUAKE_II/'),
 (0.3845285925737313,
  'https://store.steampowered.com/app/422970/Devil_Daggers/'),
 (0.3804661819941902,
  'https://store.steampowered.com/app/2280/Ultimate_Doom/'),
 (0.37359124128459775,
  'https://store.steampowered.com/app/222480/Resident_Evil_Revelations__Biohazard_Revelations/'),
 (0.3662367388016208,
  'https://store.steampowered.com/app/208200/Doom_3_BFG_Edition/'),
 (0.36613925662655805,
  'https://store.steampowered.com/app/2200/Quake_III_Arena/'),
 (0.36528140366999, 'https://store.steampowered.com/app/2290/Final_DOOM/'),
 (0.36505371164211614,
  'https://store.steampowered.com/app/41000/Serious_Sam_HD_The_First_E

In [132]:
results['https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/']

[(1.0000000000000002,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.9118261072490957,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.8202120666747894, 'https://store.steampowered.com/app/40300/Risen/'),
 (0.8162473352047446,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.8014873221180177,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.7983823107438658,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.7544879141307598, 'https://store.steampowered.com/app/39500/Gothic_3/'),
 (0.7507194835844215,
  'https://store.steampowered.com/app/22100/Mount__Blade/'),
 (0.7269504485788207, 'https://store.steampowered.com/app/22300/Fallout_3/'),
 (0.7186491547486452,
  'https://store.steampowered.com/app/306130/The_Elder_Scrolls_Online/'),
 (0.7151086148960484,
  'https://store.steampower

In [149]:
weighted_results['https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/']

[(0.7367817310945569,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.6533612428831993,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.5248437532543796, 'https://store.steampowered.com/app/40300/Risen/'),
 (0.49492686386352086,
  'https://store.steampowered.com/app/378648/The_Witcher_3_Wild_Hunt__Blood_and_Wine/'),
 (0.47620347484629133,
  'https://store.steampowered.com/app/22380/Fallout_New_Vegas/'),
 (0.4652511154081566,
  'https://store.steampowered.com/app/292030/The_Witcher_3_Wild_Hunt/'),
 (0.45312335250479097,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.44681002475692627,
  'https://store.steampowered.com/app/102500/Kingdoms_of_Amalur_Reckoning/'),
 (0.432584187682607,
  'https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/'),
 (0.42614434251819583,
  'https://store.steampowered.com/ap